#モジュールのインポート

In [1]:
import time
start_time = time.time()

In [2]:
#パスを通す
import os
from pathlib import Path
import sys
PROJECT_FOLDER = '/content/drive/MyDrive/enrich_me'
ORIGINAL_MODULES = PROJECT_FOLDER + '/modules'
FILE_MODULES = PROJECT_FOLDER + '/execution_scripts/current_files'
THIRDPARTY_MODULES = '/content/drive/MyDrive/python/Lib/site-packages'
for module_folder in [ORIGINAL_MODULES, FILE_MODULES, THIRDPARTY_MODULES]:
    if module_folder not in sys.path:
        sys.path.append(module_folder)
#seleniumの設定
import selenium_suite_installer
exec(selenium_suite_installer.code_obj)

from utils import Paths
import pandas as pd
from datetime import datetime

#パラメータ設定

In [3]:
'''パス類'''
NEW_SECTOR_LIST_CSV = f'{Paths.SECTOR_REDEFINITIONS_FOLDER}/New48sectors_list.csv' #別でファイルを作っておく
NEW_SECTOR_PRICE_PKLGZ = f'{Paths.SECTOR_REDEFINITIONS_FOLDER}/New48sectors_price.pkl.gz' #出力のみなのでファイルがなくてもOK
ML_DATASET_PATH = f'{Paths.ML_DATASETS_FOLDER}/New48sectors.pkl.gz'
'''ユニバースを絞るフィルタ'''
universe_filter = "(Listing==1)&((ScaleCategory=='TOPIX Core30')|(ScaleCategory=='TOPIX Large70')|(ScaleCategory=='TOPIX Mid400'))" #現行のTOPIX500
'''上位・下位何業種を取引対象とするか？'''
trading_sector_num = 3
candidate_sector_num = 5
'''トップ予想の業種にどれほどの傾斜をかけるか'''
top_slope = 1.5
'''学習期間'''
train_start_day = datetime(2014, 1, 1)
train_end_day = datetime(2021, 12, 31)
test_start_day = datetime(2014, 1, 1)
test_end_day = datetime(2099, 12, 31) #ずっと先の未来を指定
'''学習するか否か'''
should_learn = False
'''価格情報等のデータを更新するか否か'''
should_update_historical_data = None # 基本はNone, 必要時のみFalseに変更すること！

#実行

In [4]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

import os
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/enrich_me/modules/.env')

from PO48Sectors_fixed_240430 import main
ml_dataset = asyncio.run(main(ML_DATASET_PATH, NEW_SECTOR_LIST_CSV, NEW_SECTOR_PRICE_PKLGZ,
                          universe_filter, trading_sector_num, candidate_sector_num,
                          train_start_day, train_end_day, test_start_day, test_end_day,
                          top_slope, should_learn, should_update_historical_data))

/content/drive/MyDrive/enrich_me/modules/stock_dfs_processor.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  codes_to_replace_dict = {row[0]:row[1] for _, row in codes_to_replace_df.iterrows()}


2024-08-14 16:22:23.809560
予測＆新規建：False
建玉決済：False
取引情報取得：False


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


インスタンス情報を読み込みました。
インスタンス情報を読み込みました。
2024-08-14： 利益（税引前）95,021円（0.226%）


In [5]:
end_time = time.time()
print(f'実行時間: {end_time - start_time}')

実行時間: 156.6990990638733


In [6]:
import evaluate_model
model_stats = evaluate_model.LongShortModel('48Sector', ml_dataset.pred_result_df, ml_dataset.raw_target_df,
                                            start_date=datetime(2022,1,1), end_date=datetime(2024,7,31), bin_num=11,
                                            sector_to_estimate_num=trading_sector_num, top_slope=top_slope)
#top_slopeを設定したい。
model_stats.display_result()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


48Sectorの読み込みが完了しました。
全期間
母集団の推定（99.7%）
平均リターン: 0.1346% (worst: 0.071%, best: 0.1981%)
標準偏差: 0.00536 (worst:0.005844, best:0.004944)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.251043 (worst:0.12155, best:0.400646)
最大ドローダウン: 4.8037% (worst:10.8177%, best:2.7764%)
最大ドローダウン日数: 143日 (worst:609日, best:56日)
----------------------------------------------------------------------------------
直近1年
母集団の推定（99.7%）
平均リターン: 0.1031% (worst: 0.0034%, best: 0.2028%)
標準偏差: 0.005283 (worst:0.006077, best:0.004659)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.195157 (worst:0.005533, best:0.435362)
最大ドローダウン: 6.091% (worst:247.149%, best:2.408%)
最大ドローダウン日数: 236日 (worst:294006日, best:47日)
----------------------------------------------------------------------------------
直近3ヶ月
母集団の推定（99.7%）
平均リターン: 0.1388% (worst: -0.0831%, best: 0.3607%)
標準偏差: 0.005701 (worst:0.00769, best:0.004483)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.24345 (worst:-0.108088, best:0

Output()